<a href="https://colab.research.google.com/github/Degananda264/Language-Translation-using-Seq2Seq-Models/blob/master/Word_Levl_ENG_MAR_Languuage_Translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [100]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string
from string import digits
import re
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input,Dense,Embedding,LSTM
from keras.models import Model

In [101]:
lines=pd.read_table("mar.txt",names=['eng','mar','desc'])

In [102]:
lines.shape

(39961, 3)

In [103]:
lines.head(2)

,eng,mar,desc
0,Go.,जा.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
1,Run!,पळ!,CC-BY 2.0 (France) Attribution: tatoeba.org #9...


In [104]:
lines=lines.drop('desc',axis=1)

In [105]:
lines.head(3)

,eng,mar
0,Go.,जा.
1,Run!,पळ!
2,Run!,धाव!


In [106]:
# Lowercase all characters
lines.eng=lines.eng.apply(lambda x: x.lower())
lines.mar=lines.mar.apply(lambda x: x.lower())

In [107]:
# Remove quotes
lines.eng=lines.eng.apply(lambda x: re.sub("'", '', x))
lines.mar=lines.mar.apply(lambda x: re.sub("'", '', x))

In [108]:
exclude = set(string.punctuation) # Set of all special characters
# Remove all the special characters
lines.eng=lines.eng.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines.mar=lines.mar.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [109]:

# Remove all numbers from text
remove_digits = str.maketrans('', '', digits)


In [110]:
digits

'0123456789'

In [111]:
remove_digits

{48: None,
 49: None,
 50: None,
 51: None,
 52: None,
 53: None,
 54: None,
 55: None,
 56: None,
 57: None}

In [112]:
lines.eng=lines.eng.apply(lambda x: x.translate(remove_digits))
lines.mar = lines.mar.apply(lambda x: re.sub("[२३०८१५७९४६]", "", x))

In [113]:
# Remove extra spaces
lines.eng=lines.eng.apply(lambda x: x.strip())
lines.mar=lines.mar.apply(lambda x: x.strip())
lines.eng=lines.eng.apply(lambda x: re.sub(" +", " ", x))
lines.mar=lines.mar.apply(lambda x: re.sub(" +", " ", x))

In [114]:
# Add start and end tokens to target sequences
lines.mar = lines.mar.apply(lambda x : 'START_ '+ x + ' _END')

In [115]:
lines.head()

,eng,mar
0,go,START_ जा _END
1,run,START_ पळ _END
2,run,START_ धाव _END
3,run,START_ पळा _END
4,run,START_ धावा _END


In [116]:
lines.sample(10)

,eng,mar
37669,i want to talk to you about something else,START_ मला तुमच्याशी अजून कशाबद्दल तरी बोलायचं...
11111,he was very patient,START_ तो अतिशय धीराचा होता _END
24516,i havent spoken to anyone,START_ माझं कोणाशीही बोलणं झालं नाहीये _END
14094,theres a hotel here,START_ इथे एक हॉटेल आहे _END
26254,ill see you next wednesday,START_ पुढच्या बुधवारी भेटू _END
13660,im fixing something,START_ मी काहीतरी दुरुस्त करतेय _END
27472,how was your summer vacation,START_ उन्हाळ्याची सुट्टी कशी गेली _END
3625,tom wants mary,START_ टॉमला मेरी हवी आहे _END
26901,we just need thirty minutes,START_ आपल्याला फक्त तीस मिनिटांची गरज आहे _END
18417,we have a lot to learn,START_ आपल्याला अजून भरपूर काही शिकायचं आहे _END


In [117]:
#Voabulary of english
all_eng_words=set()
for eng in lines.eng:
  for word in eng.split():
    if word not in all_eng_words:
      all_eng_words.add(word)


In [118]:
len(all_eng_words)

5662

In [119]:
#vocabulary of marathi
all_mar_words=set()
for mar in lines.mar:
  for word in mar.split():
    if word not in all_mar_words:
      all_mar_words.add(word)

In [120]:
len(all_mar_words)

13614

In [121]:
# Max Length of source sequence
lenght_list=[]
for l in lines.eng:
    lenght_list.append(len(l.split(' ')))
max_length_src = np.max(lenght_list)
max_length_src

34

In [122]:

# Max Length of target sequence
lenght_list=[]
for l in lines.mar:
    lenght_list.append(len(l.split(' ')))
max_length_tar = np.max(lenght_list)
max_length_tar

37

In [123]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_mar_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_mar_words)
num_encoder_tokens, num_decoder_tokens

(5662, 13614)

In [124]:
num_decoder_tokens += 1 # For zero padding
num_decoder_tokens


13615

In [125]:
input_token_index=dict([(word,idx+1) for idx,word in enumerate(input_words)])

In [128]:
input_token_index

{'a': 1,
 'abbreviation': 2,
 'ability': 3,
 'able': 4,
 'about': 5,
 'above': 6,
 'abroad': 7,
 'absent': 8,
 'absolute': 9,
 'absolutely': 10,
 'accent': 11,
 'accept': 12,
 'acceptable': 13,
 'accepted': 14,
 'access': 15,
 'accessible': 16,
 'accident': 17,
 'accidentally': 18,
 'accomplices': 19,
 'according': 20,
 'accordion': 21,
 'account': 22,
 'accused': 23,
 'accusing': 24,
 'accustomed': 25,
 'ace': 26,
 'aces': 27,
 'ache': 28,
 'aches': 29,
 'achille': 30,
 'achilles': 31,
 'acid': 32,
 'acknowledged': 33,
 'acquired': 34,
 'acres': 35,
 'across': 36,
 'act': 37,
 'acted': 38,
 'acting': 39,
 'action': 40,
 'activist': 41,
 'actor': 42,
 'actors': 43,
 'actress': 44,
 'acts': 45,
 'actual': 46,
 'actually': 47,
 'add': 48,
 'added': 49,
 'addict': 50,
 'addicted': 51,
 'addition': 52,
 'additional': 53,
 'addon': 54,
 'address': 55,
 'administrative': 56,
 'admire': 57,
 'admit': 58,
 'admitted': 59,
 'adopt': 60,
 'adopted': 61,
 'ads': 62,
 'adultery': 63,
 'advance': 6

In [129]:
target_token_index=dict([(word,idx+1)for idx,word in enumerate(target_words)])

In [130]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [131]:
reverse_target_char_index

{1: '10',
 2: '100',
 3: '300',
 4: '3ds',
 5: 'START_',
 6: '_END',
 7: 'a',
 8: 'b',
 9: 'h₂o',
 10: 'ntt',
 11: 'tatoebaorg',
 12: 'uk',
 13: 'अ',
 14: 'अँकरेजमार्गे',
 15: 'अँजिलीस',
 16: 'अँटीव्हायरस',
 17: 'अँड',
 18: 'अँडी',
 19: 'अँडीजमधल्या',
 20: 'अँब्युलन्स',
 21: 'अंक',
 22: 'अंकांनी',
 23: 'अंकारा',
 24: 'अंकारात',
 25: 'अंकारामध्ये',
 26: 'अंग',
 27: 'अंगठ्या',
 28: 'अंगाला',
 29: 'अंगोला',
 30: 'अंघोळ',
 31: 'अंठ्यांपेक्षा',
 32: 'अंड',
 33: 'अंडरवुडचा',
 34: 'अंडरवेअर',
 35: 'अंडी',
 36: 'अंडींचा',
 37: 'अंडीसुद्धा',
 38: 'अंड्यांवर',
 39: 'अंत',
 40: 'अंतर',
 41: 'अंतराळयात्री',
 42: 'अंतराळयान',
 43: 'अंतराळयानाचं',
 44: 'अंतराळयानातून',
 45: 'अंतराळवीर',
 46: 'अंतराळात',
 47: 'अंतरित',
 48: 'अंतिम',
 49: 'अंदाज',
 50: 'अंदाजे',
 51: 'अंधार',
 52: 'अंधाराची',
 53: 'अंधारात',
 54: 'अंमलात',
 55: 'अंशांकित',
 56: 'अकरा',
 57: 'अकरावीत',
 58: 'अकल्पनीय',
 59: 'अकाउंट',
 60: 'अकाउंटमध्ये',
 61: 'अकॉर्डियन',
 62: 'अक्कलदाढ',
 63: 'अक्वॅरियममध्ये',
 64: 'अक्षर',
 65: 'अक्षर

In [132]:
lines=shuffle(lines)

In [133]:
lines.head(10)

,eng,mar
11420,i was in my bedroom,START_ मी माझ्या बेडरूममध्ये होतो _END
33029,there is milk in the refrigerator,START_ फ्रिजमध्ये दूध आहे _END
36386,can you play the guitar yes i can,START_ तुला गिटार वाजवता येते का हो येते _END
12310,tom made a sandwich,START_ टॉमने सँडविच बनवलं _END
11207,i can take tom home,START_ मी टॉमला घरी घेऊन जाऊ शकतो _END
38129,assuming it rains tomorrow what should i do,START_ समजा उद्या पाऊस पडला तर मी करू _END
31470,tom is coming with us to boston,START_ टॉम आमच्याबरोबर बॉस्टनला येतोय _END
3217,im behind you,START_ मी तुझ्या पाठी आहे _END
25971,how many pens does she have,START_ त्यांच्याकडे किती पेन आहे _END
4459,im very clever,START_ मी खूप हुशार आहे _END


In [134]:
# Train - Test Split
X, y = lines.eng, lines.mar
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1)
X_train.shape, X_test.shape

((35964,), (3997,))

**Save the train and test dataframes for reproducing the results later, as they are shuffled.**

In [135]:
X_train.to_pickle('/X_train.pkl')
X_test.to_pickle('/X_test.pkl')

In [136]:
batch_size=128
encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')

In [137]:
encoder_input_data.shape

(128, 34)

In [138]:
decoder_input_data.shape

(128, 37)

In [139]:
decoder_target_data.shape

(128, 37, 13615)

In [140]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

**Encoder-Decoder Architecture**

In [141]:
latent_dim=50
#Encoder
encoder_inputs=Input(shape=(None,))
enc_emb=Embedding(num_encoder_tokens,latent_dim,mask_zero=True)(encoder_inputs)
encoder_lstm=LSTM(latent_dim,return_state=True)
encoder_outputs,state_h,state_c=encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states=[state_h,state_c]

#Decoder
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['acc'])
model.summary()



Model: "model_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_19 (InputLayer)           (None, None)         0                                            
__________________________________________________________________________________________________
input_20 (InputLayer)           (None, None)         0                                            
__________________________________________________________________________________________________
embedding_11 (Embedding)        (None, None, 50)     283100      input_19[0][0]                   
__________________________________________________________________________________________________
embedding_12 (Embedding)        (None, None, 50)     680750      input_20[0][0]                   
____________________________________________________________________________________________

In [142]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 128
epochs = 50


In [84]:
model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/50
280/280 [==============================] - 99s 355ms/step - loss: 0.9828 - acc: 0.1801 - val_loss: 0.8457 - val_acc: 0.1992
Epoch 2/50
280/280 [==============================] - 95s 341ms/step - loss: 0.8520 - acc: 0.2096 - val_loss: 0.8189 - val_acc: 0.2204
Epoch 3/50
280/280 [==============================] - 95s 341ms/step - loss: 0.8090 - acc: 0.2398 - val_loss: 0.7380 - val_acc: 0.2465
Epoch 4/50
280/280 [==============================] - 96s 342ms/step - loss: 0.7757 - acc: 0.2587 - val_loss: 0.7941 - val_acc: 0.2599
Epoch 5/50
280/280 [==============================] - 95s 340ms/step - loss: 0.7460 - acc: 0.2775 - val_loss: 0.7264 - val_acc: 0.2778
Epoch 6/50
280/280 [==============================] - 95s 340ms/step - loss: 0.7185 - acc: 0.2982 - val_loss: 0.7756 - val_acc: 0.2949
Epoch 7/50
280/280 [==============================] - 95s 340ms/step - loss: 0.6945 - acc: 0.3166 - val_loss: 0.6690 - val_acc: 0.3136
Epoch 8/50
280/280 [==============================] - 9

In [85]:

model.save_weights('nmt_weights.h5')

In [143]:

model.load_weights('nmt_weights.h5')

**Inference Setup**

In [144]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

**Decode sample sequeces**

In [151]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

**Evaluation on Train Dataset**

In [152]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1

In [153]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: tell me the reason why they are absent
Actual Marathi Translation:  त्या अनुपस्थित का आहेत याचं मला कारण सांग 
Predicted Marathi Translation:  त्या अनुपस्थित का आहेत का माहीत नाहीये 


In [154]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: he has gone to italy to study music
Actual Marathi Translation:  ते संगीताचा अभ्यास करण्यास इटलीला गेले आहेत 
Predicted Marathi Translation:  तो इंग्रजीचा अभ्यास करतोय 


In [155]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: you can stay with us
Actual Marathi Translation:  तू आमच्याबरोबर राहू शकतोस 
Predicted Marathi Translation:  तू आमच्याबरोबर राहू शकतेस 


In [156]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: this orange is too sour
Actual Marathi Translation:  हे संत्र खूपच आंबट आहे 
Predicted Marathi Translation:  हे खूपच नाही आहे 


In [157]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: he kept on telling lies
Actual Marathi Translation:  तो खोटं बोलत गेला 
Predicted Marathi Translation:  ते खोटं बोलत आहेत 


**Evaluation on Validation Dataset**

In [158]:
val_gen = generate_batch(X_test, y_test, batch_size = 1)
k=-1

In [159]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])


Input English sentence: give it to him
Actual Marathi Translation:  ते त्यांना दे 
Predicted Marathi Translation:  त्यांना द्या 


In [160]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])


Input English sentence: we were all on the bus
Actual Marathi Translation:  आम्ही सगळे बसमध्ये होतो 
Predicted Marathi Translation:  आम्ही सर्व रडत होतो 


In [161]:

k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])



Input English sentence: tom heard mary crying
Actual Marathi Translation:  टॉमने मेरीला रडताना ऐकलं 
Predicted Marathi Translation:  टॉमने मेरीला आवाज ऐकू गेला 


In [162]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])


Input English sentence: were you here all night
Actual Marathi Translation:  तू इथे रात्रभर होतीस का 
Predicted Marathi Translation:  तू इथे रात्रभर होतास का 
